In [1]:
import pandas as pd
import numpy as np
import pickle

In [3]:
import pymrio
io = pymrio.parse_exiobase3('C://Users/Maxime/Desktop/Thesis/Bdd/EXIOBASE/monetary/exiobase3.4/IOT_2011_pxp/')

In [4]:
io.Y

region                                                                                             AT  \
category                                                  Final consumption expenditure by households   
region sector                                                                                           
AT     Paddy rice                                                                            0.000000   
       Wheat                                                                                40.724029   
       Cereal grains nec                                                                   126.366919   
       Vegetables, fruit, nuts                                                             889.708053   
       Oil seeds                                                                            51.537378   
       Sugar cane, sugar beet                                                                0.000000   
       Plant-based fibers                                                                    0.809726   
       Crops nec                                                                             0.000000   
       Cattle                                                                                0.000000   
       Pigs                                                                                  0.000000   
       Poultry                                                                             293.170392   
       Meat animals nec                                                                     66.076666   
       Animal products nec                                                                  43.015685   
       Raw milk                                                                             33.781096   
       Wool, silk-worm cocoons                                                               0.000000   
       Manure (conventional treatment)                                                       0.000000   
       Manure (biogas treatment)                                                             0.000000   
       Products of forestry, logging and related servi...                                  357.216856   
       Fish and other fishing products; services incid...                                   16.283516   
       Anthracite                                                                            0.000000   
       Coking Coal                                                                           0.000000   
       Other Bituminous Coal                                                                 0.000000   
       Sub-Bituminous Coal                                                                   0.000000   
       Patent Fuel                                                                           0.000000   
       Lignite/Brown Coal                                                                    0.000000   
       BKB/Peat Briquettes                                                                   0.000000   
       Peat                                                                                  0.014049   
       Crude petroleum and services related to crude o...                                    0.000000   
       Natural gas and services related to natural gas...                                    0.000000   
       Natural Gas Liquids                                                                   0.000000   
...                                                                                               ...   
WM     Research and development services (73)                                                0.000000   
       Other business services (74)                                                          6.326394   
       Public administration and defence services; com...                                    0.032165   
       Education services (80)                                                               0.469932   
       Hea

In [3]:
with open('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/technology_matrix.pickle','rb') as f:
    A = pd.read_pickle(f)

### For each quintile

In [4]:
df = pd.read_excel('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/Concordance.xlsx','Per_quintile_desaggregated')

In [5]:
list_quintile =['1st_quintile','2nd_quintile','3rd_quintile','4th_quintile','5th_quintile']

commodities = [i[1] for i in A.iloc[:200].index]
commodities.append('grid_mix')

# remove the Fs of the quebec stats excel
df = df.mask(df=='F')
df.loc[:,list_quintile] = df.loc[:,list_quintile].fillna(0)

for quintile in list_quintile:
    dict_ = {}
    for commodity in commodities:
        dict_[commodity] = 0
    for product in df.index:
        if product == 'Viande':
            list_viande = df.loc[product, ['EXIOBASE sector(s)','Unnamed: 6','Unnamed: 7','Unnamed: 8']].tolist()
            meat_vector = io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[list_viande]/io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[list_viande].sum()*df.loc[product,quintile]
            for meat in meat_vector.index:
                dict_[meat] = meat_vector[meat]
        elif product == 'Produits laitiers et oeufs':
            dict_['Dairy products'] = 0.93 * df.loc[product, quintile]
            dict_['Food products nec'] = 0.07 * df.loc[product, quintile]
        elif product == 'Eau, combustibles et électricité pour logement principal':
            dict_['grid_mix'] = 0.70 * df.loc[product, quintile]
            dict_['Distribution services of gaseous fuels through mains'] = 0.07 * df.loc[product, quintile]
            dict_['Heavy Fuel Oil'] = 0.08 * df.loc[product, quintile]
            dict_['Products of forestry, logging and related services (02)'] = 0.15 * df.loc[product, quintile]
        elif product == 'Automobiles, fourgonnettes et camions à usage privé':
            list_automobile_expenses = ['Motor vehicles, trailers and semi-trailers (34)','Motor Gasoline','Gas/Diesel Oil','Sale, maintenance, repair of motor vehicles, motor vehicles parts, motorcycles, motor cycles parts and accessoiries']
            for expense in list_automobile_expenses:
                dict_[expense] = (io.Y.loc[[i for i in io.Y.index if i[1] == expense],('CA','Final consumption expenditure by households')].sum()/
                                  io.Y.loc[[i for i in io.Y.index if i[1] in list_automobile_expenses],
                                           ('CA','Final consumption expenditure by households')].sum()*
                                  df.loc[[product,"Utilisation d'automobiles, de fourgonnettes et de camions"],quintile].sum())
        # we just aggregated buying cars with using cars, so pass cause it's already included
        elif product == "Utilisation d'automobiles, de fourgonnettes et de camions":
            pass
        else:
            try:
                if np.isnan(df.loc[product, 'Unnamed: 6']):
                    dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]
            except:
                # if it pops an error it means there is a sector in Unnamed: 3, meaning we have to split the purchases
                try:
                    if np.isnan(df.loc[product, 'Unnamed: 7']):
                        dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]/2
                        dict_[df.loc[product, 'Unnamed: 6']] += df.loc[product, quintile]/2
                except:
                    if np.isnan(df.loc[product, 'Unnamed: 8']):
                        dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]/3
                        dict_[df.loc[product, 'Unnamed: 6']] += df.loc[product, quintile]/3
                        dict_[df.loc[product, 'Unnamed: 7']] += df.loc[product, quintile]/3
    
    # add Canadian data for waste management triggered by households consumption
    waste_treatment_sectors = [i[1] for i in io.A.index if ('waste for treatment' in i[1] or 'landfill' in i[1] or 'sludge' in i[1]) and i[0] == 'CA']
    serie = (io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[waste_treatment_sectors]/
            io.Y.loc[:,('CA','Final consumption expenditure by households')].sum().sum()*sum(dict_.values()))
    for index in serie.index:
        dict_[index] = serie.loc[index]
    
    # copy canadian consumption and scale the imports to the consumption of one household from Quebec
    dff = io.Y.loc[:, ('CA','Final consumption expenditure by households')].copy()
    total_commodity = {}
    foo = dff.groupby('sector').sum()
    for index in foo.index:
        total_commodity[index] = foo[index]
    for index in dff.index:
        if total_commodity[index[1]] != 0:
            dff[index] = dff[index] / total_commodity[index[1]] * dict_[index[1]]

    # just add 201 products for Quebec, to distinguish them from CA
    Y = pd.DataFrame(dff)
    Y.columns = ['Households Quebec']
    df_to_add = Y.loc[[i for i in Y.index if i[0] == 'CA']]
    df_to_add = df_to_add.append(pd.DataFrame(dict_['grid_mix'],index=[('QC','grid_mix')],columns=['Households Quebec']))
    df_to_add = df_to_add.set_index(pd.MultiIndex.from_product([['QC'],commodities]))
    Y = pd.concat([Y, df_to_add])
    Y.loc[[i for i in dff.index if i[0] == 'CA']] = 0

    with open('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/final_demand_'+str(quintile)+'.pickle','wb') as f:
        pickle.dump(Y, f)

We assume that all products are bought and fabricated in Quebec. There is no interaction between Quebec's consumption and other provinces. Reasonable because inter-provincial tarde is negligible (1% of total imports): http://www.stat.gouv.qc.ca/statistiques/economie/commerce-exterieur/imp_pays.htm

### For average

In [ ]:
df = pd.read_excel('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/Concordance.xlsx','Average')

In [ ]:
# Fuels and meat are evenly distributed (not optimal, check methodology for each quintile and adapt if needed)

commodities = [i[1] for i in A.iloc[:200].index]
commodities.append('grid_mix')

dict_ = {}
for commodity in commodities:
    dict_[commodity] = 0

for product in df.index:
    try:
        if np.isnan(df.loc[product, 'Unnamed: 3']):
            dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, '2011 amount ($)']
    except:
        # if it pops an error it means there is a sector in Unnamed: 3, meaning we have to split the purchases
        try:
            if np.isnan(df.loc[product, 'Unnamed: 4']):
                dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, '2011 amount ($)']/2
                dict_[df.loc[product, 'Unnamed: 3']] += df.loc[product, '2011 amount ($)']/2
        except:
            # same logic
            try:
                if np.isnan(df.loc[product, 'Unnamed: 5']):
                    dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, '2011 amount ($)']/3
                    dict_[df.loc[product, 'Unnamed: 3']] += df.loc[product, '2011 amount ($)']/3
                    dict_[df.loc[product, 'Unnamed: 4']] += df.loc[product, '2011 amount ($)']/3
            except:
                # and one last round
                    dict_[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, '2011 amount ($)']/4
                    dict_[df.loc[product, 'Unnamed: 3']] += df.loc[product, '2011 amount ($)']/4
                    dict_[df.loc[product, 'Unnamed: 4']] += df.loc[product, '2011 amount ($)']/4
                    dict_[df.loc[product, 'Unnamed: 5']] += df.loc[product, '2011 amount ($)']/4
df = io.Y.loc[:, ('CA','Final consumption expenditure by households')].copy()
total_commodity = {}
foo = df.groupby('sector').sum()
for index in foo.index:
    total_commodity[index] = foo[index]

for index in df.index:
    if total_commodity[index[1]] != 0:
        df[index] = df[index] / total_commodity[index[1]] * dict_[index[1]]

Y = pd.DataFrame(df)
Y.columns = ['Households Quebec']
df_to_add = Y.loc[[i for i in Y.index if i[0] == 'CA']]
df_to_add = df_to_add.append(pd.DataFrame(dict_['grid_mix'],index=[('QC','grid_mix')],columns=['Households Quebec']))
df_to_add = df_to_add.set_index(pd.MultiIndex.from_product([['QC'],commodities]))
Y = pd.concat([Y, df_to_add])
Y.loc[[i for i in df.index if i[0] == 'CA']] = 0

with open('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/final_demand_average.pickle','wb') as f:
    pickle.dump(Y, f)